<a href="https://colab.research.google.com/github/gcasaldi/CS50/blob/main/Un_benvenuto_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
!pip install openpyxl
import pandas as pd
from datetime import datetime, timedelta
from google.colab import files
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from collections import Counter





In [83]:
from google.colab import files
import zipfile
import os

# Carica il file ZIP
uploaded = files.upload()

# Verifica il nome del file caricato
file_name = list(uploaded.keys())[0]
zip_path = f'/content/{file_name}'

# Estrai il contenuto del file ZIP
extracted_folder = '/content/extracted/'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

# Verifica i file estratti
extracted_files = os.listdir(extracted_folder)
print(f"File estratti: {extracted_files}")

# Salva il path del file Excel estratto per usarlo nella seconda cella
excel_file_path = os.path.join(extracted_folder, extracted_files[0])
print(f"File Excel estratto: {excel_file_path}")



Saving it-superenalotto-past-draws-archive.zip to it-superenalotto-past-draws-archive.zip
File estratti: ['it-superenalotto-past-draws-archive.xls']
File Excel estratto: /content/extracted/it-superenalotto-past-draws-archive.xls


In [84]:
import pandas as pd

# Usa il path ottenuto dalla prima cella
file_path = '/content/extracted/it-superenalotto-past-draws-archive.xls'  # Assicurati che questo percorso sia corretto

def carica_database(file_path):
    # Usa 'xlrd' come engine per i file .xls
    df = pd.read_excel(file_path, engine='xlrd', skiprows=1)
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')
    numeri_estratti = numeri_estratti.tail(500)
    return numeri_estratti

# Carica i dati nel DataFrame
df = carica_database(file_path)

# Visualizza le prime 10 righe
print(df.head(10))


*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
   Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7
0         NaN         NaN         NaN         NaN         NaN
1         NaN         NaN         NaN         NaN         NaN
2         2.0         3.0         4.0         5.0         6.0
3        16.0        35.0        59.0        65.0        87.0
4        24.0        25.0        30.0        43.0        47.0
5        10.0        18.0        19.0        32.0        36.0
6        28.0        52.0        63.0        77.0        79.0
7        12.0        33.0        40.0        41.0        83.0
8        12.0        37.0        59.0        63.0        90.0
9        31.0        33.0        42.0        51.0        72.0


Codice per ordinare e correggere i dati:

In [85]:
# Pulizia dei dati: rimuoviamo le colonne vuote e manteniamo solo i numeri
def pulizia_dati(df):
    df_cleaned = df.select_dtypes(include='number').dropna(axis=1, how='all')  # Manteniamo solo colonne con numeri
    return df_cleaned

# Applichiamo la pulizia
df_cleaned = pulizia_dati(df)

# Mostra le prime 10 righe dopo la pulizia
print(df_cleaned.head(10))


   Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7
0         NaN         NaN         NaN         NaN         NaN
1         NaN         NaN         NaN         NaN         NaN
2         2.0         3.0         4.0         5.0         6.0
3        16.0        35.0        59.0        65.0        87.0
4        24.0        25.0        30.0        43.0        47.0
5        10.0        18.0        19.0        32.0        36.0
6        28.0        52.0        63.0        77.0        79.0
7        12.0        33.0        40.0        41.0        83.0
8        12.0        37.0        59.0        63.0        90.0
9        31.0        33.0        42.0        51.0        72.0


Seleziona solo le colonne numeriche (i numeri estratti)

In [86]:
# Seleziona solo le colonne numeriche (i numeri estratti)
df_cleaned = df.select_dtypes(include=[np.number])

# Rimuove righe che contengono tutti valori NaN
df_cleaned = df_cleaned.dropna(how='all')

# Converte i numeri in interi per eliminare i .0
df_cleaned = df_cleaned.astype(int)

# Stampa i primi numeri per verificare
print(df_cleaned.head(10))

    Unnamed: 3  Unnamed: 4  Unnamed: 5  Unnamed: 6  Unnamed: 7
2            2           3           4           5           6
3           16          35          59          65          87
4           24          25          30          43          47
5           10          18          19          32          36
6           28          52          63          77          79
7           12          33          40          41          83
8           12          37          59          63          90
9           31          33          42          51          72
10          58          68          83          89          90
11          40          48          49          52          89



# Prendi solo le ultime 500 estrazioni

In [88]:
df_cleaned = df_cleaned.tail(800)

# Normalizza i dati tra 0 e 1

In [89]:
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df_cleaned)



# Funzione per creare sequenze di dati

In [90]:
def prepara_dati(df_scaled, sequenza_lunghezza=5):
    X, y = [], []
    for i in range(len(df_scaled) - sequenza_lunghezza):
        X.append(df_scaled[i:i + sequenza_lunghezza])
        y.append(df_scaled[i + sequenza_lunghezza])  # Previsione successiva
    return np.array(X), np.array(y)


# Prepara i dati per il modello LSTM

In [112]:
def prepara_dati(df_scaled, sequenza_lunghezza=5):
    X, y = [], []
    for i in range(len(df_scaled) - sequenza_lunghezza):  # Modifica qui
        X.append(df_scaled[i:i + sequenza_lunghezza])  # Aggiungi sequenza di lunghezza 5
        y.append(df_scaled[i + sequenza_lunghezza])  # Previsione successiva
    return np.array(X), np.array(y)


Creiamo il modello LSTM e lo alleniamo

In [113]:
# Prepara i dati
X, y = prepara_dati(df_scaled)

# Dividi i dati in set di training e testing
split_index = int(len(X) * 0.8)  # Usa l'80% per il training
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Crea il modello LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))  # Output layer con numero di unità pari a features

# Compila il modello
model.compile(optimizer='adam', loss='mean_squared_error')

# Allena il modello
model.fit(X_train, y_train, epochs=100, batch_size=32)  #epochs=50, batch_size=32

# Valuta il modello
loss = model.evaluate(X_test, y_test)
print(f"Loss sul set di test: {loss}")

# Fai delle previsioni
predictions = model.predict(X_test)

# Inverti la normalizzazione per ottenere le previsioni nella scala originale
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

# Stampa le prime 10 previsioni
print("Prime 10 previsioni:")
print(predictions[:10])

# Stampa le prime 10 osservazioni reali
print("\nPrime 10 osservazioni reali:")
print(y_test[:10])



Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 0.4807
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.4516
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 0.4218
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.3953
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.3680
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.3451
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.3193
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.2940
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.2633
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.2390
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.2281
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1894
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.1677
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.1382
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 0.1136
Epoch 16/100
1/1 ━━━━

Ottimizzazione del modello mix tra dati statistici e training

In [128]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

# Funzione per calcolare la similarità tra la previsione e l'estrazione
def calcola_similarita(previsione, estrazione):
    return len(set(previsione) & set(estrazione))

# Funzione per testare diverse combinazioni di pesi
def ottimizza_pesi(previsione_ml, top_frequenti, ritardatari, estrazioni_reali):
    best_weight_model = 0
    best_weight_stat = 0
    best_precision = 0

    # Prova diverse combinazioni di pesi (grid search)
    for weight_model in np.arange(0, 1.1, 0.1):
        for weight_stat in np.arange(0, 1.1, 0.1):
            if weight_model + weight_stat > 1:  # La somma dei pesi non deve superare 1
                continue

            previsione_combinata = []

            # Aggiungi le previsioni del modello ponderate
            for num in previsione_ml:
                if num not in previsione_combinata and len(previsione_combinata) < 6:
                    previsione_combinata.append(num)

            # Aggiungi i numeri frequenti ponderati
            for num in top_frequenti:
                if num not in previsione_combinata and len(previsione_combinata) < 6:
                    previsione_combinata.append(num)

            # Aggiungi i numeri ritardatari ponderati
            for num in ritardatari[:3]:
                if num not in previsione_combinata and len(previsione_combinata) < 6:
                    previsione_combinata.append(num)

            # Se non raggiungi 6 numeri, aggiungi casualmente
            while len(previsione_combinata) < 6:
                random_number = np.random.choice(ritardatari)  # Aggiungi un numero ritardatario casuale
                if random_number not in previsione_combinata:
                    previsione_combinata.append(random_number)

            # Calcola la similarità tra la previsione e le estrazioni reali
            total_similarity = 0
            for estrazione in estrazioni_reali:
                total_similarity += calcola_similarita(previsione_combinata, estrazione)

            # Calcola la precisione media
            precisione = total_similarity / len(estrazioni_reali)

            # Aggiorna la miglior combinazione di pesi
            if precisione > best_precision:
                best_precision = precisione
                best_weight_model = weight_model
                best_weight_stat = weight_stat

    return best_weight_model, best_weight_stat, best_precision


# Test su un sottoinsieme di estrazioni reali (ultime 50 estrazioni)
estrazioni_reali = df.tail(50).values.tolist()  # Usa le ultime 50 estrazioni

# Ottimizzazione dei pesi
best_weight_model, best_weight_stat, best_precision = ottimizza_pesi(
    predictions[-1].astype(int),  # Predizioni del modello
    [num for num, freq in numeri_ordinati_frequenza[:5]],  # Top 5 numeri frequenti
    ritardatari,  # Numeri ritardatari
    estrazioni_reali
)

print(f"\nMigliori pesi trovati: modello={best_weight_model}, statistica={best_weight_stat}")
print(f"Precisione media ottenuta: {best_precision:.2f}")

# Previsione finale con i pesi ottimizzati
previsione_combinata_ottimizzata = []

# Aggiungi le previsioni del modello ponderate
for num in predictions[-1].astype(int):
    if num not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num)

# Aggiungi i numeri più frequenti ponderati
for num in numeri_ordinati_frequenza[:5]:
    if num[0] not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num[0])

# Aggiungi i numeri ritardatari ponderati
for num in ritardatari[:3]:
    if num not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num)

# Se non raggiungi 6 numeri, prendi numeri casuali dai ritardatari
while len(previsione_combinata_ottimizzata) < 6:
    random_number = np.random.choice(ritardatari)  # Aggiungi un numero ritardatario casuale
    if random_number not in previsione_combinata_ottimizzata:
        previsione_combinata_ottimizzata.append(random_number)

# Stampa la previsione finale ottimizzata
print(f"\nPrevisione finale ottimizzata: {previsione_combinata_ottimizzata}")



Migliori pesi trovati: modello=0.0, statistica=0.0
Precisione media ottenuta: 0.57

Previsione finale ottimizzata: [10, 18, 15, 31, 72.0, 40.0]


In [136]:
# Previsione finale ottimizzata: senza duplicati e con numeri interi
previsione_combinata_ottimizzata = []

# Aggiungi la percentuale di ottimizzazione
percentuale_model = peso_model_ottimizzato * 100  # Pesi ottimizzati per il modello, espressi come percentuali
percentuale_stat = peso_stat_ottimizzato * 100  # Pesi ottimizzati per la statistica, espressi come percentuali

# Prendi l'ultima estrazione
ultima_estrazione = df_cleaned.tail(1).values.flatten()  # Consideriamo l'ultima estrazione, se necessario puoi regolare questo

# Aggiungi le previsioni ponderate (senza duplicati)
previsione_combinata_ottimizzata.extend([int(num) for num in predictions[-1].astype(int)[:int(6 * peso_model_ottimizzato)]])

# Aggiungi i numeri frequenti ponderati (senza duplicati)
previsione_combinata_ottimizzata.extend([num for num, _ in numeri_ordinati_frequenza[:int(6 * peso_stat_ottimizzato)]])

# Aggiungi numeri ritardatari se necessario (senza duplicati)
for num in ritardatari[:3]:
    # Check if num is not NaN before appending
    if not pd.isnull(num) and num not in previsione_combinata_ottimizzata and len(previsione_combinata_ottimizzata) < 6:
        previsione_combinata_ottimizzata.append(num)

# Elimina duplicati dalla previsione finale
previsione_combinata_ottimizzata = list(set(previsione_combinata_ottimizzata))

# Rimuovi i numeri che sono già usciti nell'ultima estrazione
previsione_combinata_ottimizzata = [num for num in previsione_combinata_ottimizzata if num not in ultima_estrazione]

# Se ci sono meno di 6 numeri, aggiungi numeri mancanti dalla lista dei numeri ritardatari
while len(previsione_combinata_ottimizzata) < 6:
    for rit in ritardatari:
        # Check if rit is not NaN before appending
        if not pd.isnull(rit) and rit not in previsione_combinata_ottimizzata:
            previsione_combinata_ottimizzata.append(rit)
        if len(previsione_combinata_ottimizzata) == 6:
            break

# Convertili tutti in interi
previsione_combinata_ottimizzata = [int(num) for num in previsione_combinata_ottimizzata]

# Stampa la previsione finale con la percentuale di ottimizzazione
print(f"\nPrevisione finale ottimizzata:")
print(f"Numeri previsti: {previsione_combinata_ottimizzata}")
print(f"Percentuale ottimizzata per il modello: {percentuale_model:.2f}%")
print(f"Percentuale ottimizzata per la statistica: {percentuale_stat:.2f}%")



Previsione finale ottimizzata:
Numeri previsti: [72, 40, 10, 41, 18, 2]
Percentuale ottimizzata per il modello: 50.00%
Percentuale ottimizzata per la statistica: 50.00%


Previsione e salvataggio in CSV

In [114]:
import numpy as np
import pandas as pd

# Predizione della prossima estrazione
ultima_sequenza = X[-1].reshape(1, X.shape[1], X.shape[2])
predizione = modello.predict(ultima_sequenza)

# Denormalizza la predizione per riportare i valori tra 1 e 90
predizione_denorm = scaler.inverse_transform(predizione)

# Arrotonda e limita i valori tra 1 e 90
def trasforma_predizione(predizione_denorm):
    predizione_arrotondata = np.round(predizione_denorm).astype(int)  # Arrotonda e converte in int
    predizione_arrotondata = np.clip(predizione_arrotondata, 1, 90)  # Assicura i numeri validi
    return predizione_arrotondata

# Applica la trasformazione
predizione_interi = trasforma_predizione(predizione_denorm)

# Controllo prima di salvare
print(f"Predizione della prossima estrazione: {predizione_interi.flatten()}")

# Creazione del DataFrame con il formato corretto
df_predizione = pd.DataFrame(predizione_interi, columns=["Num1", "Num2", "Num3", "Num4", "Num5"])

# Salvataggio corretto in CSV
csv_path = "/content/prossima_estrazione.csv"
df_predizione.to_csv(csv_path, index=False, header=True)

print(f"File CSV salvato correttamente in {csv_path}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
Predizione della prossima estrazione: [1 1 1 1 1]
File CSV salvato correttamente in /content/prossima_estrazione.csv


<ipython-input-114-0635bc2e6773>:13: RuntimeWarning: invalid value encountered in cast
  predizione_arrotondata = np.round(predizione_denorm).astype(int)  # Arrotonda e converte in int


In [137]:

import pandas as pd
import datetime
import zipfile

file_path = '/content/it-superenalotto-past-draws-archive.zip'

def carica_database(file_path):
    # Estrai il file .xls dal file .zip
    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        xls_file = zip_ref.namelist()[0]  # Ottieni il nome del file .xls all'interno dello zip
        with zip_ref.open(xls_file) as file:
            # Leggi il file Excel usando il file estratto
            df = pd.read_excel(file, engine='xlrd', skiprows=1)
    # Prende solo numeri e rimuove colonne vuote
    numeri_estratti = df.select_dtypes(include='number').dropna(axis=1, how='all')

    # Usa solo le ultime 50 estrazioni 🔥
    numeri_estratti = numeri_estratti.tail(50)
    return numeri_estratti

def calcola_frequenze(df):
    frequenze = df.stack().value_counts()
    return frequenze.head(5).index.tolist()

def calcola_ritardatari(df):
    numeri_totali = set(range(1, 91))
    ultime_uscite = {}

    for numero in numeri_totali:
        try:
            ultima_pos = df.apply(lambda colonna: numero in colonna.values).iloc[::-1].idxmax()
            ultime_uscite[numero] = ultima_pos
        except:
            ultime_uscite[numero] = -1

    ritardatari = sorted(ultime_uscite, key=ultime_uscite.get, reverse=True)[:5]
    return ritardatari

def somma_90(previsione):
    somma = (previsione[0] + previsione[1]) % 90
    if somma == 0:
        somma = 90
    return somma

# Esegui tutto
df = carica_database(file_path)
numeri_frequenti = calcola_frequenze(df)
numeri_ritardatari = calcola_ritardatari(df)
previsione = numeri_frequenti + numeri_ritardatari
somma = somma_90(previsione)

print(f"Data aggiornamento: {datetime.date.today()}")
print("Numeri più frequenti:", numeri_frequenti)
print("Numeri ritardatari:", numeri_ritardatari)
print("Previsione Finale:", previsione)
print("🔥 Somma 90:", somma)



*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
Data aggiornamento: 2025-03-10
Numeri più frequenti: [72.0, 77.0, 40.0, 41.0, 85.0]
Numeri ritardatari: [1, 6, 8, 9, 13]
Previsione Finale: [72.0, 77.0, 40.0, 41.0, 85.0, 1, 6, 8, 9, 13]
🔥 Somma 90: 59.0


Previsione Finale


In [138]:
import pandas as pd
import numpy as np

# Caricamento delle ultime 10 estrazioni
file_path = '/content/it-superenalotto-past-draws-archive.zip'
df = carica_database(file_path)  # Funzione che abbiamo già definito
ultime_estrazioni = df.tail(200).values  # Prendiamo solo i valori numerici

# Previsioni ottenute
previsione_ml = [26, 36, 57, 70, 90]  # Sostituisci con la tua previsione ML
top_numeri_stat = [72, 41, 40, 77, 83, 1, 6, 8, 9, 13]  # Previsione statistica

# Funzione per contare i match tra previsione e ultime estrazioni
def conta_match(previsione, estrazioni):
    match_per_estrazione = [len(set(previsione) & set(estr)) for estr in estrazioni]
    totale_match = sum(match_per_estrazione)
    return totale_match, match_per_estrazione

# Confronto tra previsioni e ultime estrazioni
match_ml, dettagli_ml = conta_match(previsione_ml, ultime_estrazioni)
match_stat, dettagli_stat = conta_match(top_numeri_stat, ultime_estrazioni)

# Analisi frequenze
frequenze = df.stack().value_counts().head(15)  # Prendi i 15 numeri più frequenti

# Output risultati
print(f"🎯 Confronto con le ultime 10 estrazioni:")
print(f"✔️ Match con ML: {match_ml} numeri trovati")
print(f"✔️ Match con Statistica: {match_stat} numeri trovati")
print("\n🔍 Dettaglio delle corrispondenze per ogni estrazione:")
print(f"ML: {dettagli_ml}")
print(f"Statistica: {dettagli_stat}")
print("\n🔥 Numeri più frequenti nelle ultime estrazioni:")
print(frequenze)

# Suggerimento per la prossima previsione
suggeriti = set(frequenze.index[:5]) | set(top_numeri_stat[:5])  # Unione dei top numeri
print("\n📌 Numeri suggeriti per la prossima estrazione:")
print(suggeriti)



*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'
🎯 Confronto con le ultime 10 estrazioni:
✔️ Match con ML: 7 numeri trovati
✔️ Match con Statistica: 35 numeri trovati

🔍 Dettaglio delle corrispondenze per ogni estrazione:
ML: [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
Statistica: [0, 0, 1, 0, 0, 0, 1, 3, 0, 1, 1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 0, 3, 3, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1]

🔥 Numeri più frequenti nelle ultime estrazioni:
72.0    9
77.0    6
40.0    6
41.0    6
85.0    5
63.0    5
83.0    5
30.0    5
29.0    5
59.0    4
87.0    4
25.0    4
49.0    4
74.0    4
86.0    4
Name: count, dtype: int64

📌 Numeri suggeriti per la prossima estrazione:
{72.0, 41.0, 40.0, 77.0, 83, 85.0}


In [48]:
import numpy as np

# Funzione migliorata per preparare i dati (controllo dei NaN)
def prepara_dati(df_cleaned, sequenza_lenght=5):
    sequenze = []
    target = []

    for i in range(len(df_cleaned) - sequenza_lenght):
        # Estrazione dei numeri per la sequenza
        sequenza = df_cleaned.iloc[i:i + sequenza_lenght].values.flatten()

        # Se la sequenza contiene NaN, la scartiamo
        if np.any(np.isnan(sequenza)):
            continue

        sequenze.append(sequenza)
        target.append(df_cleaned.iloc[i + sequenza_lenght].values.flatten())

    X = np.array(sequenze)
    y = np.array(target)

    return X, y

# Prepara i dati
X, y = prepara_dati(df_cleaned)

# Verifica la forma dei dati
print(f"Forma di X: {X.shape}")
print(f"Forma di y: {y.shape}")


Forma di X: (33, 25)
Forma di y: (33, 5)


Terza Cella: Prepara i dati per l'allenamento

In [49]:
import numpy as np

# Funzione per preparare i dati: crea sequenze temporali
def prepara_dati(df_cleaned, sequenza_lenght=5):
    sequenze = []
    target = []

    # Cicliamo per creare le sequenze
    for i in range(len(df_cleaned) - sequenza_lenght):
        sequenza = df_cleaned.iloc[i:i + sequenza_lenght].values.flatten()
        sequenze.append(sequenza)
        target.append(df_cleaned.iloc[i + sequenza_lenght].values.flatten())

    # Convertiamo in array numpy
    X = np.array(sequenze)
    y = np.array(target)

    return X, y

# Prepara i dati per l'allenamento
X, y = prepara_dati(df_cleaned)

# Mostra la forma di X e y per verificarne la corretta preparazione
print(f"Forma di X: {X.shape}")
print(f"Forma di y: {y.shape}")


Forma di X: (35, 25)
Forma di y: (35, 5)


Quarta Cella: Creazione e allenamento del modello

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Creazione del modello di rete neurale
model = Sequential()
model.add(Dense(128, input_dim=X.shape[1], activation='relu'))  # Primo strato
model.add(Dense(64, activation='relu'))  # Secondo strato
model.add(Dense(y.shape[1], activation='linear'))  # Strato di uscita per predizioni numeriche

# Compilazione del modello con una funzione di perdita appropriata per la regressione
model.compile(optimizer=Adam(), loss='mean_squared_error', metrics=['accuracy'])

# Allenamento del modello
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

# Salviamo il modello allenato per usi futuri
model.save("/content/lotto_model.h5")



Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_loss: nan
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.0000e+00 - loss: nan - val_accuracy: 0.0000e+00 - val_los

In [51]:
# Se X è 2D, aggiungiamo una dimensione per le caratteristiche
if len(X.shape) == 2:
    X = np.expand_dims(X, axis=-1)  # Aggiungiamo una dimensione per le features

print(X.shape)  # Ora X dovrebbe essere (n_samples, sequence_length, 1)

# Creazione del modello LSTM
modello = Sequential()
modello.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
modello.add(LSTM(units=50))
modello.add(Dense(units=5))  # Predizione di 5 numeri per ogni sequenza

# Compilazione del modello
modello.compile(optimizer=Adam(), loss='mean_squared_error')

# Addestramento del modello
modello.fit(X, y, epochs=10, batch_size=32)

# Predizione della prossima estrazione
predizione = modello.predict(X[-1].reshape(1, X.shape[1], X.shape[2]))  # Utilizza l'ultima sequenza per la previsione

# Funzione per trasformare la previsione in numeri interi tra 1 e 90
def trasforma_predizione(predizione):
    predizione_arrotondata = np.round(predizione)  # Arrotonda i valori continui
    predizione_arrotondata = np.clip(predizione_arrotondata, 1, 90)  # Limita i valori tra 1 e 90
    return predizione_arrotondata.astype(int)  # Converte in interi

# Trasforma la predizione in numeri interi da 1 a 90
predizione_interi = trasforma_predizione(predizione)
# Se X è 2D, aggiungiamo una dimensione per le caratteristiche
if len(X.shape) == 2:
    X = np.expand_dims(X, axis=-1)  # Aggiungiamo una dimensione per le features

print(X.shape)  # Ora X dovrebbe essere (n_samples, sequence_length, 1)

# Creazione del modello LSTM
modello = Sequential()
modello.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
modello.add(LSTM(units=50))
modello.add(Dense(units=5))  # Predizione di 5 numeri per ogni sequenza

# Compilazione del modello
modello.compile(optimizer=Adam(), loss='mean_squared_error')

# Addestramento del modello
modello.fit(X, y, epochs=10, batch_size=32)

# Predizione della prossima estrazione
predizione = modello.predict(X[-1].reshape(1, X.shape[1], X.shape[2]))  # Utilizza l'ultima sequenza per la previsione

# Funzione per trasformare la previsione in numeri interi tra 1 e 90
def trasforma_predizione(predizione):
    predizione_arrotondata = np.round(predizione)  # Arrotonda i valori continui
    predizione_arrotondata = np.clip(predizione_arrotondata, 1, 90)  # Limita i valori tra 1 e 90
    return predizione_arrotondata.astype(int)  # Converte in interi

# Trasforma la predizione in numeri interi da 1 a 90
predizione_interi = trasforma_predizione(predizione)

print(f"Predizione della prossima estrazione: {predizione_interi}")

(35, 25, 1)
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - loss: nan
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: nan
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: nan
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan 
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: nan 
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: nan 


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
(35, 25, 1)
Epoch 1/10


<ipython-input-51-eb2057967d88>:26: RuntimeWarning: invalid value encountered in cast
  return predizione_arrotondata.astype(int)  # Converte in interi


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - loss: nan
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: nan 
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan 
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: nan 
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: nan
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: nan


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 309ms/step
Predizione della prossima estrazione: [[-9223372036854775808 -9223372036854775808 -9223372036854775808
  -9223372036854775808 -9223372036854775808]]


<ipython-input-51-eb2057967d88>:55: RuntimeWarning: invalid value encountered in cast
  return predizione_arrotondata.astype(int)  # Converte in interi


Quinta Cella: Fare previsioni

noooo

next step

In [ ]:
# Genera una previsione (esempio semplificato)
# La previsione si basa sui numeri più frequenti
previsione = numeri_frequenti

NameError: name 'numeri_frequenti' is not defined

In [ ]:
# Crea un DataFrame per la previsione
previsione_df = pd.DataFrame({'Numero': previsione})


In [ ]:
# Salva la previsione in un file CSV
previsione_df.to_csv('/content/previsione_lotto.csv', index=False)

print("Previsione salvata in /content/previsione_lotto.csv")

from google.colab import files
files.download('/content/previsione_lotto.csv')


Previsione salvata in /content/previsione_lotto.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>